In [1]:
import pandas as pd
import requests
from scrapy.http import TextResponse
import time
import numpy as np
import re 

In [2]:
url="http://books.toscrape.com/"
all_urls=['http://books.toscrape.com/catalogue/page-{}.html'.format(i) for i in range(1,51)]

### Task 1. Collect	the	following	information	from	all	of	the	pages	(and	all	of	the	books)	on	books.toscrape.com:	

a)	Name	of	the	book,
b)	Price,
c)	Rating.

- Note	1:	complete	the	task	twice	using	CSS	Selector	and	Xpath	approaches.
- Note	2:	use	functions	and	loops	whenever	applicable.
- Note	3:	you	may	develop	your	code	without	functions	in	the	beginning,	and	add	functions	only	if	you	see	it	works.

## Task 1. Xpath

In [3]:
#name, price and ratings of the books
def book_scraper(URL):
    page=requests.get(URL)
    response=TextResponse(url=page.url, body=page.text,encoding="utf-8")
    books=response.xpath('//h3/a/@title').extract()
    prices=response.xpath('//p[@class="price_color"]/text()').extract()
    ratings=response.xpath('//p[contains(@class,"star-rating")]/@class').extract()
    return books, prices,ratings

In [4]:
all_book_names=[]
all_prices=[]
all_ratings=[]
for i in all_urls:
    all_book_names.extend(book_scraper(i)[0])
    all_prices.extend(book_scraper(i)[1])
    all_ratings.extend(book_scraper(i)[2])
    time.sleep(2)
print(all_book_names, all_prices,all_ratings)

['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind', 'The Requiem Red', 'The Dirty Little Secrets of Getting Your Dream Job', 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'The Black Maria', 'Starving Hearts (Triangular Trade Trilogy, #1)', "Shakespeare's Sonnets", 'Set Me Free', "Scott Pilgrim's Precious Little Life (Scott Pilgrim #1)", 'Rip it Up and Start Again', 'Our Band Could Be Your Life: Scenes from the American Indie Underground, 1981-1991', 'Olio', 'Mesaerion: The Best Science Fiction Stories 1800-1849', 'Libertarianism for Beginners', "It's Only the Himalayas", 'In Her Wake', 'How Music Works', 'Foolproof Preserving: A Guide to Small Batch Jams, Jellies, Pickles, Condiments, and More: A Foolproof Guide to Making Small Batch Jams, Jellies, Pickles, Condiments, and More',

## Task.1 CSS Selectors

In [5]:
def book_scraper2(url):
    page=requests.get(url)
    response=TextResponse(url=page.url, body=page.text,encoding="utf-8" )
    books=response.css("h3>a::attr(title)").extract()
    prices=response.css("p[class='price_color']::text").extract()
    ratings=response.css('.star-rating').xpath('./@class').extract()
    return books, prices, ratings

In [6]:
all_book_names2=[]
all_prices2=[]
all_ratings2=[]
for i in all_urls:
    all_book_names2.extend(book_scraper2(i)[0])
    all_prices2.extend(book_scraper2(i)[1])
    all_ratings2.extend(book_scraper2(i)[2])
    time.sleep(2)
print(all_book_names2[0:10], all_prices2[0:10],all_ratings2[0:10])

['A Light in the Attic', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History of Humankind', 'The Requiem Red', 'The Dirty Little Secrets of Getting Your Dream Job', 'The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull', 'The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics', 'The Black Maria'] ['Â£51.77', 'Â£53.74', 'Â£50.10', 'Â£47.82', 'Â£54.23', 'Â£22.65', 'Â£33.34', 'Â£17.93', 'Â£22.60', 'Â£52.15'] ['star-rating Three', 'star-rating One', 'star-rating One', 'star-rating Four', 'star-rating Five', 'star-rating One', 'star-rating Four', 'star-rating Three', 'star-rating Four', 'star-rating One']


### Task 2. Answer	the	following	questions	(show	your	work):
- What	is	the	most	popular	rating?
- How	many	books	on	the	website	have	that	rating?
- What	is	the	average	price	of	his/her	boos?

In [7]:
#Task2_1, 2_2
number_of_stars=[all_ratings[i].split(' ')[1] for i in range(0,len(all_ratings))]
print(number_of_stars[0:10])

['Three', 'One', 'One', 'Four', 'Five', 'One', 'Four', 'Three', 'Four', 'One']


In [8]:
#counting the most popular rating
[[i,number_of_stars.count(i)] for i in set(number_of_stars)]

[['Four', 179], ['Three', 203], ['Five', 196], ['One', 226], ['Two', 196]]

<b> The most popular rating is One. 226 books on the website have rating of One.</b>

In [9]:
#Task 2_3
all_prices=[all_prices[i].split('Â£')[1] for i in range(0,len(all_ratings))]
all_prices=np.array(all_prices).astype(np.float)
all_prices[0:5]

array([51.77, 53.74, 50.1 , 47.82, 54.23])

In [10]:
rating_price=pd.DataFrame(list(zip(number_of_stars,all_prices)), columns=("stars","prices"))
rating_price.head()

,stars,prices
0,Three,51.77
1,One,53.74
2,One,50.10
3,Four,47.82
4,Five,54.23


In [11]:
rating_price.prices[rating_price.stars=="One"].mean()

34.5611946902655

<b>Average price of the books w/ rating One is 34.561 pounds.</b>

### Task 3. Scrape	all	the	book	descriptions	from	the	website.	Suggested	workflow:
- a)	define	a	function	that	scrapes	all	the	individual	book	page	URLs
- b)	define	a	function	which	takes	individual	page URL	as	an	input and	scrapes	description.
- c)	use	function	1	to	get	all	URLs	and	function.2	to	get	all	descriptions from those	URLs.

In [12]:
def page_scrape(url):
    page=requests.get(url)
    response=TextResponse(url=page.url, body=page.text,encoding="utf-8")
    names=response.xpath('//h3/a/@href').extract()
    return names

In [13]:
page_scrape('http://books.toscrape.com/catalogue/page-1.html')

['a-light-in-the-attic_1000/index.html',
 'tipping-the-velvet_999/index.html',
 'soumission_998/index.html',
 'sharp-objects_997/index.html',
 'sapiens-a-brief-history-of-humankind_996/index.html',
 'the-requiem-red_995/index.html',
 'the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'the-black-maria_991/index.html',
 'starving-hearts-triangular-trade-trilogy-1_990/index.html',
 'shakespeares-sonnets_989/index.html',
 'set-me-free_988/index.html',
 'scott-pilgrims-precious-little-life-scott-pilgrim-1_987/index.html',
 'rip-it-up-and-start-again_986/index.html',
 'our-band-could-be-your-life-scenes-from-the-american-indie-underground-1981-1991_985/index.html',
 'olio_984/index.html',
 'mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html',

In [14]:
#urls of all books from all pages
page_urls=[]
for i in all_urls:
    for j in range(0,len(page_scrape(i))):
              page_urls.append('http://books.toscrape.com/catalogue/'+page_scrape(i)[j])

In [15]:
page_urls[0:10] #these urls will be used to scrape the description of each book

['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html',
 'http://books.toscrape.com/catalogue/the-requiem-red_995/index.html',
 'http://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html',
 'http://books.toscrape.com/catalogue/the-coming-woman-a-novel-based-on-the-life-of-the-infamous-feminist-victoria-woodhull_993/index.html',
 'http://books.toscrape.com/catalogue/the-boys-in-the-boat-nine-americans-and-their-epic-quest-for-gold-at-the-1936-berlin-olympics_992/index.html',
 'http://books.toscrape.com/catalogue/the-black-maria_991/index.html']

In [16]:
def desc_scrape(URL):
    page=requests.get(URL)
    response=TextResponse(url=page.url, body=page.text,encoding="utf-8" ) 
    description=response.xpath('//p[normalize-space(.//text())]/text()').extract()[1]
    return description

In [18]:
#let's get the description of all books
all_descriptions=[]
for i in range(0,len(page_urls)):
    all_descriptions.append(desc_scrape(page_urls[i]))
    time.sleep(1)
print(all_descriptions[0:5]) #first 5 book descriptions have been printed to save the space:D

["It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone down here'sGot it in for you. Shel, you never sound

### Task 4. Answer	the	following	questions	(most	probably	using	RegEx):
- a)	How	many	books	have	a	numeric	character	in	title?
- b)	How	many	books	have	a	year	in	title?
- c)	How	many	books	have	an	enumerated	character	(starting	with	#	and	followed	by	a	number)	in	title?

In [20]:
p = re.compile('\d+')
nums=[]
for i in all_book_names:
    nums.append(p.findall(i))
nums=[i for i in nums if i] #removing empty lists

In [21]:
nums[0:10]

[['1936'],
 ['1'],
 ['1'],
 ['1981', '1991'],
 ['1800', '1849'],
 ['2'],
 ['1883', '1993'],
 ['10'],
 ['1'],
 ['2014']]

In [22]:
len(nums)

365

#### There are 365 titles w/ numbers.

In [23]:
years=[]
for i in nums:
    for j in i:
        if len(j)>=4:
            years.append(i)

In [24]:
years

[['1936'],
 ['1981', '1991'],
 ['1981', '1991'],
 ['1800', '1849'],
 ['1800', '1849'],
 ['1883', '1993'],
 ['1883', '1993'],
 ['2014'],
 ['1980'],
 ['1491'],
 ['1', '2014', '2015', '1'],
 ['1', '2014', '2015', '1']]

In [25]:
len(years)

12

#### There are 12 books with a year in title.

In [27]:
p = re.compile('#(\d+)')
numbers=[]
for i in all_book_names:
    numbers.append(p.findall(i))
len([i for i in numbers if i])

313

#### There are 313 books an	enumerated	character	(starting	with	#	and	followed	by	a	number).